In [ ]:
"""
This script runs an evaluation of a LangGraph agent on a dataset using LangSmith's async evaluation API.
It converts dataset examples into the agent's internal state format, invokes the agent graph with unique checkpointing,
and uses a custom evaluator that leverages an LLM judge to compare the agent's output against reference answers.

Key components:
- example_to_state: converts dataset inputs to the agent's state format.
- target_with_config: async wrapper to invoke the agent graph with checkpoint config.
- correct: async evaluator function that prompts an LLM to judge correctness of the agent's answer.
- aevaluate: runs the evaluation over the dataset with concurrency and experiment tracking.
"""

#==============================================================================
# IMPORTS
#==============================================================================
import uuid
from langsmith import aevaluate
from my_agent import create_graph
from my_agent.utils.state import DataAnalysisState
from my_agent.utils.models import get_azure_llm

#==============================================================================
# INITIALIZATION
#==============================================================================
# Get Model
judge_llm = get_azure_llm(temperature=0.0)

# Create the agent graph
graph = create_graph()

#==============================================================================
# EVALUATION FUNCTIONS
#==============================================================================
async def correctness(outputs: dict, reference_outputs: dict) -> bool:
    """
    Evaluator function that uses an LLM to determine if the actual answer contains all information
    from the expected answer.

    Args:
        outputs (dict): The output dictionary from the agent, expected to contain a 'messages' list.
        reference_outputs (dict): The reference outputs from the dataset, expected to have 'answers' key.

    Returns:
        bool: True if the LLM judge determines the answer is correct, False otherwise.
    """
    if not outputs or "messages" not in outputs or not outputs["messages"]:
        return False
        
    actual_answer = outputs["messages"][-1].content
    expected_answer = reference_outputs.get("answers", "[NO EXPECTED ANSWER PROVIDED]")
    
    instructions = (
        "Given an actual answer and an expected answer, determine whether"
        " the actual answer contains all of the information in the"
        " expected answer. Respond with 'CORRECT' if the actual answer"
        " does contain all of the expected information and 'INCORRECT'"
        " otherwise. Do not include anything else in your response."
    )
    user_msg = f"ACTUAL ANSWER: {actual_answer}\n\nEXPECTED ANSWER: {expected_answer}"
    
    response = await judge_llm.ainvoke(
        [
            {"role": "system", "content": instructions},
            {"role": "user", "content": user_msg}
        ]
    )
    return response.content.upper() == "CORRECT"

#==============================================================================
# STATE CONVERSION FUNCTIONS
#==============================================================================
def example_to_state(inputs: dict) -> dict:
    """
    Converts dataset example inputs into the agent's internal DataAnalysisState format.

    Args:
        inputs (dict): Input dictionary from the dataset, expected to have 'question' key.

    Returns:
        DataAnalysisState: Initialized state object for the agent.
    """
    return DataAnalysisState(
        prompt=inputs['question'],
        messages=[],
        result="",
        iteration=0
    )

async def target_with_config(inputs: dict):
    """
    Async wrapper to invoke the agent graph with a unique checkpoint thread_id in the config.

    Args:
        inputs (dict): Input dictionary from the dataset.

    Returns:
        dict: The output from the agent graph invocation, including 'messages'.
    """
    state = example_to_state(inputs)
    config = {"configurable": {"thread_id": str(uuid.uuid4())}}
    return await graph.ainvoke(state, config=config)

#==============================================================================
# MAIN EVALUATION
#==============================================================================
experiment_results = await aevaluate(
    target_with_config,
    data="czsu agent",
    evaluators=[correctness],
    max_concurrency=4,
    experiment_prefix="test1",
)

print(f"Evaluation results: {experiment_results}")


View the evaluation results for experiment: 'test1-7094f3af' at:
https://smith.langchain.com/o/817120e4-bc86-484a-9c30-795e42f5bb9e/datasets/e585a306-7113-491d-ba41-bc03c09eb612/compare?selectedSessions=8e727836-2395-445f-be13-1b083a34b661




0it [00:00, ?it/s]

3: Enter get_schema_node
3: Schema details appended
3: Enter get_schema_node
3: Schema details appended
3: Enter get_schema_node
3: Schema details appended
3: Enter get_schema_node
3: Schema details appended
4: Enter query_gen_node
4: Enter query_gen_node
4: Enter query_gen_node
4: Enter query_gen_node
4: Query generated and appended
4: Query generated: df[(df["ČR, kraje"] == "Hlavní město Praha") & (df["Kumulace čtvrtletí"] == "Q1-Q3 2024") & (df["Ukazatel"] == "Počet obyvatel na konci období - celkem")]["value"].sum() - df[(df["ČR, kraje"] == "Hlavní město Praha") & (df["Kumulace čtvrtletí"] == "Q1-Q3 2024") & (df["Ukazatel"] == "Počet obyvatel na počátku období - celkem")]["value"].sum()
9: Deciding next step (iteration 0)
9: Proceeding to correct query
5: Enter check_query_node
4: Query generated and appended
4: Query generated: df[(df["ČR, kraje"] == "Hlavní město Praha") & (df["Kumulace čtvrtletí"] == "Q1-Q3 2024") & (df["Ukazatel"] == "Počet obyvatel na konci období - celkem")][

1it [00:04,  4.20s/it]

3: Enter get_schema_node
3: Schema details appended
4: Enter query_gen_node
5: Checked query appended
5: Checked query: df[(df["Kumulace čtvrtletí"] == "Q1-Q3 2024") & (df["ČR, kraje"].isin(["Hlavní město Praha", "Středočeský kraj"])) & (df["Ukazatel"] == "Počet obyvatel na počátku období - celkem")].groupby("ČR, kraje")["value"].sum()
6: Enter execute_query_node (iteration 0)
20: Loading data from: E:\OneDrive\Knowledge Base\0207_GenAI\Code\LangGraph\testing_prototypes\prototype4\data\OBY01PDT01.csv
20: Data loaded, shape: (135, 4)
6: Executing query: df[(df["Kumulace čtvrtletí"] == "Q1-Q3 2024") & (df["ČR, kraje"].isin(["Hlavní město Praha", "Středočeský kraj"])) & (df["Ukazatel"] == "Počet obyvatel na počátku období - celkem")].groupby("ČR, kraje")["value"].sum()
20: _run called with query: df[(df["Kumulace čtvrtletí"] == "Q1-Q3 2024") & (df["ČR, kraje"].isin(["Hlavní město Praha", "Středočeský kraj"])) & (df["Ukazatel"] == "Počet obyvatel na počátku období - celkem")].groupby("ČR, 

...

3: Enter get_schema_node
3: Schema details appended
4: Enter query_gen_node


Error running target function: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-05-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}
Traceback (most recent call last):
  File "c:\Users\retko\anaconda3\envs\prototype4a\Lib\site-packages\langsmith\evaluation\_arunner.py", line 1244, in _aforward
    await fn(
  File "c:\Users\retko\anaconda3\envs\prototype4a\Lib\site-packages\langsmith\run_helpers.py", line 524, in async_wrapper
    function_result = await asyncio.create_task(  # type: ignore[call-arg]
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\retko\AppData\Local\Temp\ipykernel_31116\411

4: Query generated and appended
4: Query generated: df[(df["Kumulace čtvrtletí"] == "Q1-Q3 2024") & (df["ČR, kraje"] == "Hlavní město Praha") & (df["Ukazatel"] == "Počet obyvatel na konci období - muži")]["value"]
9: Deciding next step (iteration 0)
9: Proceeding to correct query
5: Enter check_query_node
5: Checked query appended
5: Checked query: df[(df["Kumulace čtvrtletí"] == "Q1-Q3 2024") & (df["ČR, kraje"] == "Hlavní město Praha") & (df["Ukazatel"] == "Počet obyvatel na konci období - muži")]["value"]
6: Enter execute_query_node (iteration 0)
20: Loading data from: E:\OneDrive\Knowledge Base\0207_GenAI\Code\LangGraph\testing_prototypes\prototype4\data\OBY01PDT01.csv
20: Data loaded, shape: (135, 4)
6: Executing query: df[(df["Kumulace čtvrtletí"] == "Q1-Q3 2024") & (df["ČR, kraje"] == "Hlavní město Praha") & (df["Ukazatel"] == "Počet obyvatel na konci období - muži")]["value"]
20: _run called with query: df[(df["Kumulace čtvrtletí"] == "Q1-Q3 2024") & (df["ČR, kraje"] == "Hlavní 

30it [02:13,  4.46s/it]

Evaluation results: <AsyncExperimentResults test1-7094f3af>
